In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.preprocessing import StandardScaler
import lightgbm as lgb

plt.rcParams['figure.figsize'] = (15.0, 10.0)

In [2]:
data_chemicals = pd.read_csv("chemicals.csv", encoding = "utf-8")
data_drought = pd.read_csv("droughts.csv", encoding = "utf-8")
data_education = pd.read_csv("education_attainment.csv", encoding = "latin1")

## Pollution index by county

In [3]:
data_chemicals = data_chemicals[data_chemicals['year']==2016]

In [4]:
data_chemicals = data_chemicals.set_index(data_chemicals['fips'])

In [18]:
data_chemicals['greater_than_MCL'] = (data_chemicals['contaminant_level'] == "Greater than MCL") * 1.
groups = data_chemicals.groupby("cws_name")[['greater_than_MCL', 'fips', "pop_served"]].mean()
groups['index_with_weights'] = groups['greater_than_MCL']*groups['pop_served']
groups['fips'] = groups['fips'].astype(int)
chemicals_index = groups.groupby("fips")['index_with_weights'].sum() / groups.groupby("fips")['pop_served'].sum()

## Drought index by county

In [ ]:
data_drought['valid_start'] = pd.to_datetime(data_drought['valid_start'])
data_drought['valid_end'] = pd.to_datetime(data_drought['valid_end'])

In [ ]:
data_drought['event_duration_in_days'] = (data_drought['valid_end'] - data_drought['valid_start']).dt.days

In [ ]:
data_drought['event_importance_with_weight'] = data_drought['event_duration_in_days'] * 0. + (data_drought['d0']*1. + data_drought['d1']*2. + data_drought['d2']*3. + data_drought['d3']*4. + data_drought['d4']*5.) / 100.
groups = data_drought.groupby("fips")
drought_index = groups['event_importance_with_weight'].sum() / groups['event_duration_in_days'].sum()

In [ ]:
data_drought

In [ ]:
drought_index

## Education index by county

In [ ]:
data_education

In [ ]:
data_education['avg_higher_education_years'] = (data_education['pct_less_than_hs'] * (-3) + data_education['pct_hs_diploma'] * 0. + data_education['pct_college_or_associates']*3. + data_education['pct_college_bachelors_or_higher']*4.)/100.

In [ ]:
education_index = data_education.groupby('fips')['avg_higher_education_years'].mean()

In [ ]:
education_index

In [ ]:
len(data_chemicals['fips'].unique())

In [ ]:
chemicals_index

## Analysis

In [ ]:
data_with_indices = pd.DataFrame({'drought': drought_index, 'chemicals': chemicals_index, 'education': education_index})

In [ ]:
data_with_indices

In [ ]:
data_with_indices = data_with_indices.dropna()

In [ ]:
data_with_indices['drought_normalized'] = (data_with_indices['drought'] - data_with_indices['drought'].mean()) /data_with_indices['drought'].std()
data_with_indices['chemicals_normalized'] = (data_with_indices['chemicals'] - data_with_indices['chemicals'].mean()) /data_with_indices['chemicals'].std()

In [ ]:
sns.scatterplot(data_with_indices['drought_normalized'].values, data_with_indices['chemicals_normalized'].values)
plt.grid()

In [ ]:
kmeans = KMeans(n_clusters=3)
kmeans.fit(data_with_indices[['drought_normalized', 'chemicals_normalized']].values)

In [ ]:
kmeans.labels_

In [ ]:
sns.scatterplot(data_with_indices['drought_normalized'].values, data_with_indices['chemicals_normalized'].values, c=kmeans.labels_)
plt.grid()

In [ ]:
LABEL_COLOR_MAP = {0: 'r',
                   1: 'g',
                   2: 'b'}

label_color = [LABEL_COLOR_MAP[l] for l in kmeans.labels_]
plt.scatter(data_with_indices['drought_normalized'].values, data_with_indices['chemicals_normalized'].values, c=label_color)
plt.grid()

In [ ]:
data_with_indices[['chemicals', 'drought']].corr()

In [ ]:
sns.scatterplot(data_with_indices['chemicals'].values, data_with_indices['drought'].values)
plt.grid()

## Regression chemicals vs. industry total revenue

In [ ]:
data_industry_occupation = pd.read_csv("industry_occupation.csv", encoding="latin1")
data_industry_earnings = pd.read_csv("earnings.csv", encoding="latin1")

In [ ]:
data_industry_earnings = data_industry_earnings[data_industry_earnings['year'] == 2016]
data_industry_occupation = data_industry_occupation[data_industry_occupation['year'] == 2016]

In [ ]:
data_industry_occupation = data_industry_occupation.set_index(data_industry_occupation['fips'])
data_industry_earnings = data_industry_earnings.set_index(data_industry_earnings['fips'])

In [ ]:
sector_corres = {'agri_fish_hunt': ['agriculture'],
                #'mining_quarrying_oilgas_extract': ['mining'],
                'construction': ['construction'],
                'manufacturing': ['manufacturing'],
                'wholesale_trade': ['wholesale_trade'],
                'retail_trade': ['retail_trade'],
                'transport_warehouse_utilities': ['transport_utilities'],
                'information': ['information'],
                'fin_ins_realest': ['finance_insurance_realestate'],
                'total_prof_sci_mgmt_admin': ['prof_scientific_waste', 'public_admin'],
                'total_edu_health_social': ['edu_health'],
                'total_arts_ent_acc_food': ['arts_recreation'],
                'other_ser': ['other']}

In [ ]:
data_sectors = pd.DataFrame()
for sector_earnings in sector_corres:
        print(sector_earnings)
        data_sectors[sector_earnings] = data_industry_earnings[sector_earnings] * data_industry_occupation[sector_corres[sector_earnings][0]]
        for supp_sector in sector_corres[sector_earnings][1:]:
            print('supp', supp_sector)
            data_sectors[sector_earnings] = data_industry_earnings[sector_earnings] * data_industry_occupation[supp_sector]

In [ ]:
sectors_occupation = ['agriculture', 'construction', 'manufacturing', 'wholesale_trade', 'retail_trade', 'transport_utilities', 'information', 'finance_insurance_realestate', 'prof_scientific_waste', 'edu_health', 'arts_recreation', 'other', 'public_admin']
industry_occ_pop = data_industry_occupation[sectors_occupation]
industry_occ_pop['chemicals'] = chemicals_index
industry_occ_pop['total_employed'] = data_industry_occupation['total_employed']
industry_occ_pop = industry_occ_pop.dropna()

for col in sectors_occupation:
    industry_occ_pop[col] = 100. * industry_occ_pop[col]/industry_occ_pop['total_employed']

X = industry_occ_pop[sectors_occupation]
Y = industry_occ_pop['chemicals']

In [ ]:
H = lgb.LGBMRegressor()
H.fit(X, Y)

In [ ]:
lgb.plot_importance(H)

In [ ]:
sectors_occupation

In [ ]:
# S = StandardScaler()
# X = S.fit_transform(X.values)

In [ ]:
L = Ridge()
L.fit(X, Y.values.reshape(-1,1))

In [ ]:
ind = np.arange(len(L.coef_.flatten()))
width = 0.35
fig, ax = plt.subplots()
ax.bar(ind+width, L.coef_.flatten())
ax.set_xticks(ind+width/2.)
ax.set_xticklabels(sectors_occupation)
plt.xticks(rotation=45)
plt.grid()
plt.ylabel("Weight")
plt.show()

## Regression sur les earnings

In [ ]:
data_industry_earnings.columns

In [ ]:
sectors_earnings = ['total_agri_fish_mine', 'construction', 'manufacturing', 'wholesale_trade', 'retail_trade', 'transport_warehouse_utilities', 'information', 'fin_ins_realest', 'total_prof_sci_mgmt_admin', 'total_edu_health_social', 'total_arts_ent_acc_food', 'other_ser', 'pub_admin']

for sector in sectors_earnings:
    if(data_industry_earnings[sector].dtype != float):
        data_industry_earnings[sector] = data_industry_earnings[sector].str.replace(',','')
        data_industry_earnings[sector] = data_industry_earnings[sector].astype(float)

industry_earnings = data_industry_earnings[sectors_earnings]
industry_earnings['chemicals'] = chemicals_index
industry_earnings['total_med'] = data_industry_earnings['total_med']

industry_earnings = industry_earnings.dropna()

X = industry_earnings[sectors_earnings]
Y = industry_earnings['chemicals']

In [ ]:
H = lgb.LGBMRegressor()
H.fit(X, Y)

In [ ]:
lgb.plot_importance(H)

In [ ]:
L = Ridge()
L.fit(X.values, Y.values.reshape(-1,1))

In [ ]:
ind = np.arange(len(L.coef_.flatten()))
width = 0.35
fig, ax = plt.subplots()
ax.bar(ind+width, L.coef_.flatten())
ax.set_xticks(ind+width/2.)
ax.set_xticklabels(sectors_earnings)
plt.xticks(rotation=45)
plt.grid()
plt.ylabel("Weight")
plt.show()

## Which sectors are most correlated with water pollution?

In this section, we provide insights on the link between the activity of various industry sectors and the level of water pollution in a large number of counties across the US.

### How can we measure water pollution?

The U.S. CDC regularly measures the concentration of a number of chemicals in water systems across the US. Based on these measurements for the years 2000-2016, we build a water pollution index using the following method:

For each chemical, we differentiate between two levels: $0$ if the current concentration of that chemical is below its median, and $1$ if it is above its median (that is to say if its current concentration is abnormally high). For a given water system, we then average these values across all chemicals, then compute an average across all the water systems in a given county, weighted by the population served by each water system.

This allows us to build a water pollution index between $0$ and $1$ for each county.